In [1]:
import getpass
import os

from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

In [3]:
from agno.models.huggingface import HuggingFace
from agno.agent import Agent
from agno.tools.duckduckgo import DuckDuckGoTools



* LLM

In [7]:
model=HuggingFace(
        id="meta-llama/Meta-Llama-3-8B-Instruct",
        api_key=os.getenv("HUGGINGFACE_API_TOKEN"),
)

* Web Searcher Agent

In [8]:
web_searcher = Agent(model=model, tools=[DuckDuckGoTools()], name="Web Searcher",  role="Searches the web for information on a topic", show_tool_calls=True)
response = web_searcher.run("Who is Thomas Edison?", markdown=False)
print(response.content)

Thomas Edison was an American inventor and businessman who developed many important technologies. He is best known for his development of the first practical incandescent light bulb, as well as the phonograph, the motion picture camera, and the alkaline battery. Edison held a record 1,093 patents in his name.


* RAG Agent

In [ ]:
from agno.models.ollama import Ollama

ollama_model=Ollama(id="llama3.2:3b")

In [ ]:
from agno.embedder.ollama import OllamaEmbedder

ollama_embedder = OllamaEmbedder(id="nomic-embed-text:latest",dimensions=768)

In [ ]:
from agno.vectordb.lancedb import LanceDb, SearchType

# Define the database URL where the vector database will be stored
db_url = "./tmp/lancedb"
vector_db = LanceDb(
    table_name="recipes",  # Table name in the vector database
    uri=db_url,  # Location to initiate/create the vector database
    embedder=ollama_embedder,  # Without using this, it will use OpenAIChat embeddings by default
    search_type=SearchType.vector,
)

In [ ]:
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
# Create a knowledge base of PDFs from URLs
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=vector_db
)
# Load the knowledge base
knowledge_base.load()

In [ ]:
# Create and use the agent
agent = Agent(model=ollama_model,
            knowledge=knowledge_base,  
              # Add a tool to search the knowledge base which enables agentic RAG.
               # This is enabled by default when `knowledge` is provided to the Agent.
            search_knowledge=True,
            show_tool_calls=True,
            debug_mode=False,
            markdown=False)